<a href="https://colab.research.google.com/github/arushi019/GA-Based-Neural-Architecture-Prediction/blob/master/GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import time
import torch.optim as optim
import random

In [0]:
batch_sz=10
epoch=10
lr=0.01
cache_cnn={}

In [5]:
transform=transforms.Compose([transforms.ToTensor()])
train=torchvision.datasets.MNIST(root='',train=True,transform=transform,download=True)
test=torchvision.datasets.MNIST(root='',train=False,transform=transform,download=True)
n_train=500
train_samp=SubsetRandomSampler(np.arange(n_train))
n_val=100
val_samp=SubsetRandomSampler(np.arange(n_train,n_train+n_val))
n_test=100
test_samp=SubsetRandomSampler(np.arange(n_test))
train_loader=torch.utils.data.DataLoader(train,batch_size=batch_sz,sampler=train_samp,num_workers=2)
test_loader=torch.utils.data.DataLoader(test,batch_size=batch_sz,sampler=test_samp,num_workers=2)
val_loader=torch.utils.data.DataLoader(train,batch_size=batch_sz,sampler=val_samp,num_workers=2)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 27977025.94it/s]                            


Extracting MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 447313.80it/s]
  1%|          | 16384/1648877 [00:00<00:11, 144154.84it/s]

Extracting MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 7562435.47it/s]                            
8192it [00:00, 189973.45it/s]


Extracting MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
def initialise_population(samples=10):
  population=[]
  for i in range(samples):
    temp=customCNN(batch_sz)
    population.append(temp) 
    #cache_cnn[temp.id]=1000000
  return population

In [0]:
def evaluate(net, batch_size=batch_sz, n_epochs=epoch, learning_rate=lr):
    global train_loader,test_loader,val_loader
    if cache_cnn[net.id]!=1000000:
      return cache_cnn[net.id]
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    #Get training data
    #train_loader = get_train_loader(batch_size)
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    #loss, optimizer = createLossAndOptimizer(net, learning_rate)
    loss = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
            
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            #print(outputs.shape,labels.shape)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.item()
            total_train_loss += loss_size.item()
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in val_loader:
            
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.item()
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))
    total_test_loss=0
    for inputs,labels in test_loader:
      inputs,labels=Variable(inputs),Variable(labels)
      t_outp=net(inputs)
      t_loss_size=loss(t_outp,labels)
      total_test_loss+=t_loss_size.item()
    cache_cnn[net.id]=total_test_loss/len(test_loader)
    print("Testing Loss for model",net.id,":",total_test_loss/len(test_loader))

In [0]:
def choose_best_ones(population):
  scores=[]
  for i in range(len(population)):
    try:
      evaluate(population[i])
      #scores.append(cache_cnn[population[i].id])
    except Exception as e:
      print(e)
    scores.append(cache_cnn[population[i].id])
    #print(scores[-1])
  indices=[i for i in range(len(population))]
  l=len(population)/2
  l1=len(indices)
  while(l1>l):
    i1=random.randint(0,l1-1)
    i2=random.randint(0,l1-1)
    if i1==i2:
      if i1<l1-1:
        i2=i1+1
      else:
        i2=i1-1
    print(indices,i1,i2,l1)
    if scores[indices[i1]]<scores[indices[i2]]:
      indices.remove(indices[i2])
    else:
      indices.remove(indices[i1])
    l1=l1-1
  new_parents=[population[i] for i in indices]
  return new_parents

In [0]:
def get_length(val):
    if val==1 or val==4:
      return 2
    if val==2:
      return 3
    if val==3:
      return 1
def crossover(sample1,sample2):
  i1=sample1.num
  i2=sample2.num
  n1=[int(i1/10),i1%10]
  n2=[int(i2/10),i2%10]
  n11=[n2[0],n1[1]]
  n12=[n1[0],n2[1]]
  print(n11,n12)
  t1=customCNN(batch_sz,n11)
  t2=customCNN(batch_sz,n12)
  for i in range(get_length(n11[0])):
    temp_param1=sample2.layers[i].named_parameters()
    temp_param2=t1.layers[i].named_parameters()
    d_param=dict(temp_param2)
    for name1,p1 in temp_param1:
      if name1 in d_param:
        d_param[name1].data.copy_(p1.data)
  for i in range(get_length(n12[0])):
    temp_param1=sample1.layers[i].named_parameters()
    temp_param2=t2.layers[i].named_parameters()
    d_param=dict(temp_param2)
    for name1,p1 in temp_param1:
      if name1 in d_param:
        d_param[name1].data.copy_(p1.data)
  return (t1,t2)

In [0]:
def get_children(population):
  temp=[]
  for i in range(0,len(population)-1,2):
    (t1,t2)=crossover(population[i],population[i+1])
    temp.append(t1)
    temp.append(t2)
  return temp

In [0]:
class customCNN(nn.Module):
  #                                               k x s x s         
  #block 1: conv2d+avgPool                       6 x s/2 x s/2            
  #block 2: conv2d+avgPool+dropOut               7 x s/2 x s/2            
  #block 3: reshape+linear+reshape               (1,k*s*s)->64->(1*32*32)
  #block 4: reshape+linear+dropOut+reshape       (1,k*s*s)->64->(1*32*32)
  #reshape+linear layer                          (1,k*s*s)->10
  def __init__(self,batch_sze,temp_v=None):
    super(customCNN,self).__init__()
    self.v=[0,0]
    if temp_v!=None:
      self.v[0]=temp_v[0]
      self.v[1]=temp_v[1]
    else:
      self.v[0]=np.random.randint(1,5)
      self.v[1]=np.random.randint(1,5)
    self.num=self.v[0]*10+self.v[1]
    self.id=time.time()
    cache_cnn[self.id]=1000000
    self.layers=[]
    self.batch_size=batch_sze
    k=self.batch_size
    s=28
    for i in range(2):
      val=0
      if self.v[i]==1:
        val=self.block1(k,s)
      if self.v[i]==2:
        val=self.block2(k,s)
      if self.v[i]==3:
        val=self.block3(k,s)
      if self.v[i]==4:
        val=self.block4(k,s)
      k=val[1]
      s=val[2]
      print(k,s)
      for j in range(len(val[0])):
        self.layers.append(val[0][j])
    temp_layer=nn.Linear(k*s*s,10*k)
    self.layers.append(temp_layer)
    self.layers=nn.ModuleList(self.layers)
    print(self.layers)
  def get_length(val):
    if val==1 or val==4:
      return 2
    if val==2:
      return 3
    if val==3:
      return 1
  def block1(self,k,s):
    l1=nn.Conv2d(k,6*k,kernel_size=3,stride=1,padding=1)
    l2=nn.AvgPool2d(kernel_size=2,stride=2)
    return ([l1,l2],6*k,int(s/2))
  def block2(self,k,s):
    l1=nn.Conv2d(k,7*k,kernel_size=3,stride=1,padding=1)
    l2=nn.AvgPool2d(kernel_size=2,stride=2)
    l3=nn.Dropout(p=0.5)
    return ([l1,l2,l3],7*k,int(s/2))
  def block3(self,k,s):
    #reshape in forward function
    l1=nn.Linear(k*s*s,64*k)
    #reshape 
    return ([l1],k,8)
  def block4(self,k,s):
    #reshape
    l1=nn.Linear(k*s*s,64*k)
    l2=nn.Dropout(p=0.4)
    #reshape
    return ([l1,l2],k,8)
  def ps_forward(self,x,val,index):
    #print(x.shape)
    if val==1:
      x=x.view(1,self.layers[index].in_channels,x.shape[2],-1)
      x1=F.relu(self.layers[index](x))
      x2=self.layers[index+1](x1)
      return (x2,index+2)
    if val==2:
      x=x.view(1,self.layers[index].in_channels,x.shape[2],-1)
      x1=F.relu(self.layers[index](x))
      x2=self.layers[index+1](x1)
      x3=self.layers[index+2](x2)
      return (x3,index+3)
    if val==3:
      x1=x.view(1,-1)
      x2=self.layers[index](x1)
      x3=x2.view(self.batch_size,8,-1)
      return (x3,index+1)
    if val==4:
      x1=x.view(1,-1)
      x2=self.layers[index](x1)
      x2=x2.view(1,-1)
      x3=self.layers[index+1](x2)
      x4=x3.view(self.batch_size,8,-1)
      return (x4,index+2)
  def forward(self,x):
    pt=0
    #x=x.view(1,1,x.shape[2],-1)
    #print(x.shape)
    for i in range(2):
      x,pt=self.ps_forward(x,self.v[i],pt)
      #print(x.shape)
    x=x.view(1,-1)
    x1=self.layers[pt](x)
    x1=x1.view(self.batch_size,-1)
    #x1=F.log_softmax(x1,dim=1)
    return x1

In [0]:
def best_result(population,num_epochs=100,converge=2):
  scores=[]
  for i in range(len(population)):
    try:
      evaluate(population[i])
      scores.append(cache_cnn[population[i].id])
    except Exception as e:
      print(e)
  it=0
  diff=max(scores)-min(scores)
  curr=population
  while it<num_epochs and diff>converge:
    curr=get_new_population(curr)
    scores=[]
    for i in range(len(population)):
      try:
        evaluate(population[i])
        scores.append(cache_cnn[population[i].id])
      except Exception as e:
        print(e)
    #print(scores)
    diff=max(scores)-min(scores)
    it+=1
  ind=scores.index(min(scores))
  return curr[ind]

In [0]:
def get_new_population(current):
  children=get_children(current)
  new_p=current+children
  #for i in range(len(new_p)):
  #  new_p[i]=mutation(new_p[i])
  new_p=choose_best_ones(new_p)
  return new_p

In [0]:
p=initialise_population(samples=8)
print(p)
#scores=[evaluate(i) for i in p]
scores=[]
for i in range(len(p)):
  try:
    evaluate(p[i])
    scores.append(cache_cnn[p[i].id])
  except Exception as e:
    print(e)
    
#print(scores)
sample=best_result(p,num_epochs=20,converge=0.005)
print(sample,evaluate(sample))

10 8
70 4
ModuleList(
  (0): Linear(in_features=7840, out_features=640, bias=True)
  (1): Conv2d(10, 70, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Dropout(p=0.5)
  (4): Linear(in_features=1120, out_features=700, bias=True)
)
10 8
70 4
ModuleList(
  (0): Linear(in_features=7840, out_features=640, bias=True)
  (1): Dropout(p=0.4)
  (2): Conv2d(10, 70, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (4): Dropout(p=0.5)
  (5): Linear(in_features=1120, out_features=700, bias=True)
)
60 14
60 8
ModuleList(
  (0): Conv2d(10, 60, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (2): Linear(in_features=11760, out_features=3840, bias=True)
  (3): Linear(in_features=3840, out_features=600, bias=True)
)
10 8
60 4
ModuleList(
  (0): Linear(in_features=7840, out_features=640, bias=True)
  (1): Conv2d(10, 60, kernel_siz